In [1]:
%pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [374]:
import simpy
import random
import statistics
from collections import deque
import pandas as pd
import datetime

In [365]:
# Params and constants. In minutes

#Global Simpy env variables
SEED = 1234
SIM_TIME = 2 * 30 # 8 hours of operation #can add a scalar like .5 for testing with a shorter time and to preserve the times in dict below

#Variables for project analysis
#PAX_ARRIVALS = 1/5 # The expo rate passengers arrive at the monorail station.
HEADWAY = 5 # Cyclical minutes between each train's arrival. Constant set by monorail's operators
DWELL_TIME = 3 # Minutes that a train remains in the station to allow for passenger loading. Constant set by monorail's operators
CAR_CAPACITY = 10 # Number of passengers that the park's model of trains car can hold.
CARS_PER_TRAIN = 3 # Number of train cars coupled together

# Schedule to define peak times throughout park open hours, in 30 min intervals
# Assume empiral data collection determined a peak in the morning at opening time, then again around lunch when people leave and come back. It tapers out like a tail to end the day
arrival_schedule = {1: 15, 2: 12, 3: 10, 4: 5, # First two hours, 8AM to 10AM
                    5: 3, 6: 4, 7: 3, 8: 6, # Next two hours, 10AM to 12PM
                    9: 25, 10: 22, 11: 14, 12: 8, # Next two hours, 12PM to 2PM
                    13: 6, 14: 4, 15: 3 , 16: 1# Final two hours, 2PM to4PM
                    }


In [366]:
def passenger_arrivals(env):
    """ Simpy function to generate passenger arrivals at the station platform and add to the FIFO queue"""
    pax_counter = 0

    
    while True:
        #Determine which interval the clock is in, 30 min intervals
        interval = (env.now // 30) + 1
        pax_interarrival = random.expovariate(arrival_schedule[interval]*.2) #remove .2, just slowing down for testing
        pax_counter += 1

        pax_queue.append(pax_counter)
        pax_id_list.append(pax_counter)
        pax_arrival_times.append(env.now)
        
        
        
        yield env.timeout(pax_interarrival)
        mins, seconds = divmod(env.now * 60, 60) #this time format only for printing
        print(f"Passenger {pax_counter} arrived at t={int(mins)}:{int(seconds):02d} mins")

        

In [378]:
def train_arrival(env, headway=HEADWAY, capacity=CAR_CAPACITY, n_cars=CARS_PER_TRAIN, dwell_time=DWELL_TIME):
    """ Simpy function to create the train arrivals at determined cadence"""
    total_capacity = capacity * n_cars
    train_counter = 0
    
    while True:
        
        yield env.timeout(HEADWAY)
        train_counter += 1

        #Jump into loading process function
        env.process(pax_loading(env, total_capacity, dwell_time, train_counter))
        
        mins, seconds = divmod(env.now * 60, 60)  #this time format only for printing
        print(f"=={n_cars} Car monorail with capacity {total_capacity} pax arrived at t={int(mins)}:{int(seconds):02d} mins==")

In [380]:
def pax_loading(env, total_capacity, dwell_time, train_counter):
    """ Will remove people from the queue and load the train at a stochastic rate.
    Right now i have a uniform dist for testing values. we might want to change this to something more sophisticated
    """
    # Counter to keep track of pax loaded with each call of this function
    load = 0

    # Helpful output
    if len(pax_queue) != 0: #mind the case when a 0 len queue gives an index error
        print(f"ID starting platform queue = {pax_queue[0]}")
    elif len(pax_queue) == 0:
        print(f"Waiting for passengers on platform!")
        
    print(f"==Monorail dwells and loads for max of {dwell_time} mins==")

    #Elapsed times variables
    train_arrival_time = env.now
    train_departure_time = env.now + dwell_time

    # Repeats until dwell reached
    while env.now < train_departure_time:
        # there is an edge case where the last person to board delays the train leaving. becasue it timesout while that person loads.
        
        # Condition for max capacity. ensures no more depart system
        elif load == total_capacity:
            print(f"==Monorail capacity reached! Sitting then leaving station in t={train_departure_time - env.now:.2f} mins==")
            yield env.timeout(train_departure_time - env.now) #Reaching capacity before dwell time, train must sit until set departure time

        # Condition for people up to capacity
        elif load < total_capacity:
            load_time_fake = random.uniform(0,.5) #Replace with something else, like flipped log normal or weibull
            yield env.timeout(load_time_fake)
            load += 1
            print(f"Loaded passenger n={load} @ {env.now:.2f}")

            # FIFO pop
            if len(pax_queue) != 0:               
                n = pax_queue.popleft()
                #print(f"popped {n}")
        pax_departure_times.append(env.now)
        n_trains_waited.append(train_counter)
        
    
    print(f"==Dwell reached. Loaded {load} pax. Monorail departs at t={env.now:.2f}==")
    #print("queue", pax_queue)
    if len(pax_queue) != 0:
        print(f"ID ending platform queue = {pax_queue[0]}")
    elif len(pax_queue) == 0:
        print(f"Platform cleared!")
        

In [381]:

#Stat collection data structures
#   more research to be done. just setting up a framework

pax_id_list = []
pax_arrival_times = []
pax_departure_times = []
n_trains_waited = []

pax_queue = deque() # FIFO queue to manage people arriving at and leaving in order. Stores pax id and arrival timestamp. Eg (id, time) or (4, 1.32)

#random.seed(SEED)
# or
random.seed(datetime.datetime.now().timestamp())

env = simpy.Environment()


#Processes
env.process(passenger_arrivals(env))
env.process(train_arrival(env))


env.run(until=SIM_TIME)

#Stat collection

Passenger 1 arrived at t=0:29 mins
Passenger 2 arrived at t=1:02 mins
Passenger 3 arrived at t=1:16 mins
Passenger 4 arrived at t=1:20 mins
Passenger 5 arrived at t=1:51 mins
Passenger 6 arrived at t=2:52 mins
Passenger 7 arrived at t=2:58 mins
Passenger 8 arrived at t=3:14 mins
Passenger 9 arrived at t=3:26 mins
Passenger 10 arrived at t=3:31 mins
Passenger 11 arrived at t=3:33 mins
Passenger 12 arrived at t=3:50 mins
Passenger 13 arrived at t=3:51 mins
Passenger 14 arrived at t=3:53 mins
Passenger 15 arrived at t=4:05 mins
Passenger 16 arrived at t=4:10 mins
Passenger 17 arrived at t=4:19 mins
Passenger 18 arrived at t=4:58 mins
==3 Car monorail with capacity 30 pax arrived at t=5:00 mins==
ID starting platform queue = 1
==Monorail dwells and loads for max of 3 mins==
Passenger 19 arrived at t=5:08 mins
Loaded passenger n=1 @ 5.20
Passenger 20 arrived at t=5:40 mins
Loaded passenger n=2 @ 5.68
Loaded passenger n=3 @ 5.93
Passenger 21 arrived at t=5:57 mins
Loaded passenger n=4 @ 6.10

In [368]:
print(pax_id_list)
print(pax_arrival_times)
print(pax_departure_times)
print(n_trains_waited)

print(len(pax_id_list))
print(len(pax_arrival_times))
print(len(pax_departure_times))
print(len(n_trains_waited))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186]
[0, 1.1316079356248865, 1.3253171237993824, 1.327823681152154, 2.1341066347771678, 3.0678736212683186, 3.358813097225087, 3.7299503326486727, 3.7591741588590266, 4.2440046745016

In [370]:
print(pax_queue)
print(len(pax_queue)) #Number left waiting at simlation end time
#should match this
print(len(pax_arrival_times)-len(pax_departure_times))

deque([130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186])
57
57


In [371]:
#time_data = list(zip(, , , ))

time_data = {
    'pax_id': pax_id_list,
    'pax_arrival_time': pax_arrival_times,
    'pax_departure_time': pax_departure_times,
    'n_trains_waited': n_trains_waited
}
df = pd.DataFrame.from_dict((time_data), orient='index').T
print(df)

     pax_id  pax_arrival_time  pax_departure_time  n_trains_waited
0       1.0          0.000000            5.311641              1.0
1       2.0          1.131608            5.385918              1.0
2       3.0          1.325317            5.443125              1.0
3       4.0          1.327824            5.925575              1.0
4       5.0          2.134107            6.226307              1.0
..      ...               ...                 ...              ...
181   182.0         58.125369                 NaN              NaN
182   183.0         58.178857                 NaN              NaN
183   184.0         58.560546                 NaN              NaN
184   185.0         59.395151                 NaN              NaN
185   186.0         59.713295                 NaN              NaN

[186 rows x 4 columns]
